# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import warnings
import ast
import os
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from IPython.display import display, HTML
import pickle
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## Loading of the dataset

In [2]:
credit = pd.read_csv('credits.csv')
keyword = pd.read_csv('keywords.csv')
link = pd.read_csv('links.csv')
links_small = pd.read_csv('links_small.csv')
movies_metadata = pd.read_csv('movies_metadata.csv')
ratings_small = pd.read_csv('ratings_small.csv')

In [3]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
movies_metadata.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [6]:
credit.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [7]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [8]:
keyword.head(2)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [9]:
keyword.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [10]:
ratings_small.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [11]:
ratings_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [12]:
link.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [13]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


In [14]:
links_small.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [15]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


# Simple Recommender - Popularity based

The popularity-based recommender suggests popular items to users based on their overall popularity among all users.

In [16]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [133]:
# Selecting features for the recommender.
col = ['title','release_date', 'popularity']
pop_movie = movies_metadata[col]

In [134]:
pop_movie.head()

,title,release_date,popularity
0,Toy Story,1995-10-30,21.946943
1,Jumanji,1995-12-15,17.015539
2,Grumpier Old Men,1995-12-22,11.7129
3,Waiting to Exhale,1995-12-22,3.859495
4,Father of the Bride Part II,1995-02-10,8.387519


In [135]:
# Convert the values to numeric, and then to integers
pop_movie['popularity'] = pd.to_numeric(pop_movie['popularity'], errors = 'coerce') 
pop_movie['release_date'] = pd.to_datetime(pop_movie['release_date'], errors = 'coerce') #to date format

In [21]:
pop_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         45460 non-null  object        
 1   release_date  45376 non-null  datetime64[ns]
 2   popularity    45460 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.0+ MB


In [137]:
# Popularity based on the popularity score with respect to the released year.
pop_mov = pop_movie.sort_values('popularity', ascending=False)
pop_mov = pop_mov.sort_values('release_date', ascending= False)

In [138]:
#Top 20 most popular mocies based on their respective popularity score
pop_mov = pd.DataFrame({'Recommendation':pop_mov.title.values})

In [141]:
base_url = "https://image.tmdb.org/t/p/w500"
for ind, i in enumerate(pop_mov.Recommendation.values):
    if isinstance(i, str):
        poster_path = str(movies_metadata[movies_metadata.title == i]['poster_path'].values[0])
        poster_url = base_url + poster_path
        pop_mov.loc[ind, 'poster_path'] = poster_url
    else:
        pop_mov.loc[ind, 'poster_path'] = ''

In [142]:
pop_mov.head()

,Recommendation,poster_path
0,Avatar 2,https://image.tmdb.org/t/p/w500/wvpOVl37PR21mE...
1,The Other Side of the Wind,https://image.tmdb.org/t/p/w500/wtSpgCw6MmSxGY...
2,Bad Boys for Life,https://image.tmdb.org/t/p/w500/2GwwyfykFAf3jK...
3,Mary Shelley,https://image.tmdb.org/t/p/w500/AnHutc9eDDrVXv...
4,Mobile Homes,https://image.tmdb.org/t/p/w500/of72rtl3j0C9yJ...


In [143]:
pickle.dump(pop_mov, open('title_poster_path.pkl', 'wb'))

In [3]:
title_poster_path = pickle.load(open('title_poster_path.pkl', 'rb'))
top_20 = title_poster_path[:20]
top_20

,Recommendation,poster_path
0,Avatar 2,https://image.tmdb.org/t/p/w500/wvpOVl37PR21mE...
1,The Other Side of the Wind,https://image.tmdb.org/t/p/w500/wtSpgCw6MmSxGY...
2,Bad Boys for Life,https://image.tmdb.org/t/p/w500/2GwwyfykFAf3jK...
3,Mary Shelley,https://image.tmdb.org/t/p/w500/AnHutc9eDDrVXv...
4,Mobile Homes,https://image.tmdb.org/t/p/w500/of72rtl3j0C9yJ...
5,Iron Sky: The Coming Race,https://image.tmdb.org/t/p/w500/dJTn8jUruV3llF...
6,Sly Cooper,https://image.tmdb.org/t/p/w500/rw36cctUD5kPCN...
7,The King's Daughter,https://image.tmdb.org/t/p/w500/xBLskcmADJYJrD...
8,How to Talk to Girls at Parties,https://image.tmdb.org/t/p/w500/tyCt5dQ2yhfEtW...
9,Pitch Perfect 3,https://image.tmdb.org/t/p/w500/A4bAz2rvc0vjit...


In [127]:
# Showing the poster for each recommendation
def poster(top_movie):
    html_code = '<div style="display: flex; flex-wrap: nowrap; overflow-x: auto;">'
    for poster_url in top_movie.poster_path:
        html_code += f'<img src="{poster_url}" style="max-width: 300px; margin-right: 10px;">'
    html_code += '</div>'

    display(HTML(html_code))

In [128]:
poster(top_20)

**NB:** Some of the poster path link has broken so, not all recommendation poster could be shown.

# Content based filtering

This recommender suggests similar movies to a user based on the content/features of the movies.

In [41]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [42]:
links_small.shape

(9125, 3)

In [43]:
#For removing outliers that doesn't seem like an ID such as '1998-12-23'
for ind, i in enumerate(movies_metadata['id']):
    if '-' in i:
        movies_metadata.drop(ind, inplace=True, axis=0)

In [44]:
#Converting the data type of the ID to integer form instead of i
movies_metadata['id'] = movies_metadata['id'].astype(int)

In [45]:
movies = pd.merge(movies_metadata,credit, on='id')
movies = pd.merge(movies, keyword, on= 'id')

In [46]:
movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [47]:
movies.rename(columns={'id': 'tmdbId'}, inplace=True)

In [48]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  46628 non-null  object 
 1   belongs_to_collection  4574 non-null   object 
 2   budget                 46628 non-null  object 
 3   genres                 46628 non-null  object 
 4   homepage               8009 non-null   object 
 5   tmdbId                 46628 non-null  int32  
 6   imdb_id                46611 non-null  object 
 7   original_language      46617 non-null  object 
 8   original_title         46628 non-null  object 
 9   overview               45633 non-null  object 
 10  popularity             46624 non-null  object 
 11  poster_path            46229 non-null  object 
 12  production_companies   46624 non-null  object 
 13  production_countries   46624 non-null  object 
 14  release_date           46540 non-null  object 
 15  re

In [49]:
mov_item = pd.merge(movies, links_small, on='tmdbId', how= 'inner')

In [50]:
mov_item.shape

(9219, 29)

In [51]:
mov_item.head(2)

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,movieId,imdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",2,113497


In [52]:
mov_item.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'tmdbId', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords', 'movieId',
       'imdbId'],
      dtype='object')

In [53]:
#Features for the content based filtering
selected_features = ['title','genres', 'overview', 'cast', 'crew', 'keywords'] 

In [54]:
mov_item = mov_item[selected_features]
mov_item.head(2)

,title,genres,overview,cast,crew,keywords
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [55]:
mov_item.isnull().sum()

title        0
genres       0
overview    12
cast         0
crew         0
keywords     0
dtype: int64

In [56]:
# fill with space as other features other than overview might contain important factor
mov_item['overview'] = mov_item['overview'].fillna('')

In [57]:
mov_item.isnull().sum()

title       0
genres      0
overview    0
cast        0
crew        0
keywords    0
dtype: int64

In [58]:
def convert(str_list):
# converts the string representation of a list of dictionaries into an actual list of dictionaries.
    fresh_list = []
    for i in ast.literal_eval(str_list): 
        fresh_list.append(i['name'])
    return fresh_list

In [59]:
mov_item['genres'] = mov_item['genres'].apply(convert) #getting genres of the movies
mov_item['cast'] = mov_item['cast'].apply(convert) #getting the name of the cast in the movies
mov_item['keywords'] = mov_item['keywords'].apply(convert) #keywords

In [60]:
mov_item.head()

,title,genres,overview,cast,crew,keywords
0,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva..."
1,Jumanji,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'..."
2,Grumpier Old Men,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o..."
3,Waiting to Exhale,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...","[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin..."
4,Father of the Bride Part II,[Comedy],Just when George Banks has recovered from his ...,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug..."


In [61]:
# Getting the names of director
def director_name(str_list):
    dir_list = []
    for i in ast.literal_eval(str_list):
        if i['job'] == 'Director':
            dir_list.append(i['name'])
    return dir_list

In [62]:
mov_item['crew'] = mov_item['crew'].apply(director_name) #Getting the director's name for the respective movie

In [63]:
#Splitting the words to form a list of words
mov_item['overview'] = mov_item['overview'].apply(lambda x: x.split()) 

In [64]:
mov_item.head(2)

,title,genres,overview,cast,crew,keywords
0,Toy Story,"[Animation, Comedy, Family]","[Led, by, Woody,, Andy's, toys, live, happily,...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],"[jealousy, toy, boy, friendship, friends, riva..."
1,Jumanji,"[Adventure, Fantasy, Family]","[When, siblings, Judy, and, Peter, discover, a...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],"[board game, disappearance, based on children'..."


In [65]:
mov_item['tag'] = mov_item['overview'] + mov_item['genres'] + mov_item['cast'] +  mov_item['keywords'] + mov_item['crew']
mov_item['tag'] = mov_item['tag'].apply(lambda x: [i.lower() for i in x])

In [66]:
mov_item.drop(columns=['genres', 'overview', 'cast', 'crew', 'keywords'], inplace=True)

In [67]:
mov_item.head()

,title,tag
0,Toy Story,"[led, by, woody,, andy's, toys, live, happily,..."
1,Jumanji,"[when, siblings, judy, and, peter, discover, a..."
2,Grumpier Old Men,"[a, family, wedding, reignites, the, ancient, ..."
3,Waiting to Exhale,"[cheated, on,, mistreated, and, stepped, on,, ..."
4,Father of the Bride Part II,"[just, when, george, banks, has, recovered, fr..."


In [68]:
mov_item['tag'] =  mov_item['tag'].apply(lambda x: ' '.join(x))

In [148]:
mov_item.head()

,title,tag
0,Toy Story,"led by woody, andy's toys live happily in his ..."
1,Jumanji,when siblings judy and peter discover an encha...
2,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,Waiting to Exhale,"cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,just when george banks has recovered from his ...


In [70]:
mov_item.shape

(9219, 2)

In [149]:
pickle.dump(mov_item, open('item_tag.pkl', 'wb'))

In [5]:
vectorizer = TfidfVectorizer(stop_words='english') #used for transforming text data into a numerical representation
feature_vector= vectorizer.fit_transform(item_tag['tag'])

In [6]:
similarity = cosine_similarity(feature_vector, feature_vector) #calculates the cosine similarity between pairs of vectors

In [7]:
similarity.shape

(9219, 9219)

In [8]:
similarity

array([[1.        , 0.0256714 , 0.02218016, ..., 0.        , 0.00486321,
        0.01070548],
       [0.0256714 , 1.        , 0.02745994, ..., 0.00189663, 0.01210609,
        0.01103926],
       [0.02218016, 0.02745994, 1.        , ..., 0.00199984, 0.00265029,
        0.0163416 ],
       ...,
       [0.        , 0.00189663, 0.00199984, ..., 1.        , 0.00165656,
        0.        ],
       [0.00486321, 0.01210609, 0.00265029, ..., 0.00165656, 1.        ,
        0.00140669],
       [0.01070548, 0.01103926, 0.0163416 , ..., 0.        , 0.00140669,
        1.        ]])

In [9]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [13]:
all_movies = title_poster_path.Recommendation.to_list() #list of movies title
pickle.dump(all_movies, open('all_title.pkl', 'wb'))

In [2]:
similarity = pickle.load(open('similarity.pkl', 'rb'))

In [3]:
#Splitting the data for easy upload on Github
data = similarity
num_splits = 21
split_size = data.shape[0] // num_splits

# Split the dataset into 21 parts
splits = np.split(data, num_splits)

# Save each split as a separate file
for i, split in enumerate(splits):
    file_name = f'split_{i}.pkl'
    with open(file_name, 'wb') as f:
        pickle.dump(split, f)

In [4]:
g = pickle.load(open('split_0.pkl', 'rb'))
g.shape

(439, 9219)

In [8]:
#Checking if it works
import glob
split_files = glob.glob(f'split_*.pkl')

# Load each split file and concatenate them back together
splits = []
for split_file in split_files:
    with open(split_file, 'rb') as f:
        split = pickle.load(f)
        splits.append(split)

# Concatenate the splits back into the original dataset
data = np.concatenate(splits, axis=0)

In [9]:
data.shape

(9219, 9219)

In [43]:
def movie_based_recommendation():
    item_based_rec = []
    movie_input = input('Enter the movie of your choice: ')
    match_list = []
    # Checking for the input through the title's list
    for i in all_title:
        if movie_input.lower() in str(i).lower():
            match_list.append(i)
    find_close_match = difflib.get_close_matches(movie_input, match_list) #For finding perfect match of the input title from the movie list in the dataframe.
    mov_index = item_tag[item_tag.title ==find_close_match[0]].index[0]
    distance  =  similarity[mov_index]
    movie_list = sorted(list(enumerate(distance)), key= lambda x: x[1], reverse= True)[1:21]
    
    for i in movie_list:
        mov_title = item_tag[item_tag.index ==i[0]]['title'].values[0] 
        item_based_rec.append(mov_title)
    return item_based_rec

In [44]:
content_based_output = movie_based_recommendation()

Enter the movie of your choice: rush hour


In [132]:
def mov_poster(obj):
    base_url = "https://image.tmdb.org/t/p/w500"
    html_code = '<div style="display: flex; flex-wrap: nowrap; overflow-x: auto;">'
    for i in obj:
        if isinstance(i, str):
            poster_path = str(title_poster_path[title_poster_path.Recommendation == i]['poster_path'].values[0])
            html_code += f'<img src="{poster_path}" style="max-width: 300px; margin-right: 10px;">'
    html_code += '</div>'
    
    return display(HTML(html_code))

In [46]:
top_20_rec = pd.DataFrame({'Recommend Movies':content_based_output}, index= [i for i in range(1,21)])
top_20_rec

,Recommend Movies
1,Rush Hour 3
2,Rush Hour 2
3,Police Story
4,Police Story 3: Supercop
5,Crime Story
6,New Police Story
7,First Strike
8,Project S
9,Twin Dragons
10,Once a Thief


In [85]:
mov_poster(content_based_output)

**NB:** Some poster path link has broken so, couldn't get access to it

# Collaborative Filtering Recommender System

In [90]:
movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [91]:
colab_mov = pd.merge(movies, link, on='tmdbId')

In [92]:
colab_mov.head(2)

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,movieId,imdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",2,113497


In [93]:
colab_mov.shape

(46910, 29)

In [94]:
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [95]:
ratings_small.shape

(100004, 4)

In [96]:
# Getting the smaller size of the data for the personalized system
for ind, i in enumerate(ratings_small['movieId'].values):
    colab_title = colab_mov[colab_mov.movieId == i]['title']
    if colab_title.empty == True:
        #some movies'title were missing
        ratings_small.loc[ind, 'title'] = np.nan
    else:
        ratings_small.loc[ind, 'title'] = colab_title.values[0]

In [97]:
ratings_small.head()

,userId,movieId,rating,timestamp,title
0,1,31,2.5,1260759144,Dangerous Minds
1,1,1029,3.0,1260759179,Dumbo
2,1,1061,3.0,1260759182,Sleepers
3,1,1129,2.0,1260759185,Escape from New York
4,1,1172,4.0,1260759205,Cinema Paradiso


In [98]:
ratings_small.isnull().sum()

userId         0
movieId        0
rating         0
timestamp      0
title        194
dtype: int64

In [101]:
ratings_small.dropna(axis=0, inplace=True)

In [103]:
ratings_small.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
dtype: int64

In [104]:
ratings_small.shape

(99810, 5)

In [105]:
colab_fil = pd.merge(ratings_small, links_small, on='movieId', how= 'inner')

In [106]:
colab_fil.head()

,userId,movieId,rating,timestamp,title,imdbId,tmdbId
0,1,31,2.5,1260759144,Dangerous Minds,112792,9909.0
1,7,31,3.0,851868750,Dangerous Minds,112792,9909.0
2,31,31,4.0,1273541953,Dangerous Minds,112792,9909.0
3,32,31,4.0,834828440,Dangerous Minds,112792,9909.0
4,36,31,3.0,847057202,Dangerous Minds,112792,9909.0


In [107]:
colab_features = ['userId', 'title', 'rating']
user_item_mtrx = colab_fil[colab_features]

In [108]:
user_item_mtrx.head()

,userId,title,rating
0,1,Dangerous Minds,2.5
1,7,Dangerous Minds,3.0
2,31,Dangerous Minds,4.0
3,32,Dangerous Minds,4.0
4,36,Dangerous Minds,3.0


In [109]:
user_item_mtrx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99810 entries, 0 to 99809
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   userId  99810 non-null  int64  
 1   title   99810 non-null  object 
 2   rating  99810 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.0+ MB


In [110]:
user_item = user_item_mtrx.pivot_table(index='userId', columns='title', values='rating').fillna(0)

In [161]:
user_item.head()

title,$9.99,'Neath the Arizona Skies,"'night, Mother",(500) Days of Summer,...And God Created Woman,...And Justice for All,1-900,10,10 Attitudes,10 Cloverfield Lane,10 Items or Less,10 Things I Hate About You,10 Years,"10,000 BC",100 Girls,100 Rifles,101 Dalmatians,101 Reykjavik,102 Dalmatians,11'09''01 - September 11,11:14,12 Angry Men,12 Years a Slave,127 Hours,12:08 East of Bucharest,13 Assassins,13 Ghosts,13 Going on 30,13 Tzameti,1408,15 Minutes,16 Blocks,16 Wishes,17 Again,18 Again!,1941,1984,1990: The Bronx Warriors,2 Days in New York,2 Days in the Valley,2 Fast 2 Furious,2 Guns,2 or 3 Things I Know About Her,20 Dates,20 Feet from Stardom,"20,000 Leagues Under the Sea",200 Cigarettes,200 Motels,2001: A Space Odyssey,2010,2012,2046,21,21 Grams,21 Jump Street,21 Up,22 Jump Street,23,24 Hour Party People,24: Redemption,25 Watts,25th Hour,27 Dresses,28 Days,28 Days Later,28 Up,28 Weeks Later,29th and Gay,3 Idiots,3 Ninjas,3 Ninjas Kick Back,3 Ninjas Knuckle Up,3 Ninjas: High Noon at Mega Mountain,3 Women,3-Day Weekend,3-Iron,30 Days of Night,30 Minutes or Less,30 YEARS TO LIFE,300,3000 Miles to Graceland,300: Rise of an Empire,35 Up,3:10 to Yuma,4 Little Girls,"4 Months, 3 Weeks and 2 Days",4 for Texas,40 Days and 40 Nights,42,42 Up,42nd Street,45 Years,47 Ronin,48 Hrs.,49 Up,49th Parallel,5 Centimeters per Second,50 First Dates,50/50,52 Pick-Up,54,56 Up,61*,7 Days in Hell,7 Plus Seven,71 Fragments of a Chronology of Chance,8 Heads in a Duffel Bag,8 Mile,8 Seconds,8 Women,8 ½ Women,84 Charing Cross Road,88 Minutes,8MM,8½,9,96 Minutes,99 Homes,A Beautiful Mind,A Better Tomorrow,A Boy and His Dog,A Bridge Too Far,A Brief History of Time,A Bronx Tale,A Bug's Life,A Charlie Brown Christmas,A Chinese Ghost Story,A Chinese Ghost Story II,A Chorus Line,A Christmas Carol,A Christmas Story,A Cinderella Story,A Civil Action,A Clockwork Orange,A Close Shave,A Coffee in Berlin,"A Cool, Dry Place",A Cry in the Dark,A Damsel in Distress,A Dangerous Method,A Day at the Races,A Deadly Adoption,A Decade Under the Influence,A Dirty Shame,A Dog's Life,A Dog's Will,A Double Life,A Face in the Crowd,A Family Thing,A Fantastic Fear of Everything,A Far Off Place,A Farewell to Arms,A Few Good Men,A Fish Called Wanda,A Fistful of Dollars,"A Fragile Trust: Plagiarism, Power, and Jayson Blair at the New York Times",A Friend of Mine,A Funny Thing Happened on the Way to the Forum,A Girl Walks Home Alone at Night,A Good Day to Die Hard,A Goofy Movie,A Grand Day Out,A Guide To Recognizing Your Saints,A Guy Thing,A Hard Day's Night,A History of Violence,A Home at the End of the World,A Kid in King Arthur's Court,A King in New York,A Kiss Before Dying,A Knight's Tale,A League of Ordinary Gentlemen,A League of Their Own,A Letter to Elia,A Letter to Three Wives,A Life Less Ordinary,A Little Princess,A Little Romance,A Lot Like Love,A Love Song for Bobby Long,A Low Down Dirty Shame,A Man Escaped,A Man and a Woman,A Man for All Seasons,A Map of the World,A Matter of Loaf and Death,A Midnight Clear,A Midsummer Night's Dream,A Midsummer Night's Sex Comedy,A Midwinter's Tale,A Mighty Wind,A Million Ways to Die in the West,A Month by the Lake,A Month in the Country,A Most Violent Year,A Most Wanted Man,A Night at the Opera,A Night at the Roxbury,A Night in the Life of Jimmy Reardon,A Night to Remember,A Nightmare on Elm Street,A Nightmare on Elm Street 3: Dream Warriors,A Nightmare on Elm Street 4: The Dream Master,A Nightmare on Elm Street 5: The Dream Child,A Nightmare on Elm Street Part 2: Freddy's Revenge,A Passage to India,A Patch of Blue,A Perfect Candidate,A Perfect Murder,A Perfect World,A Personal Journey with Martin Scorsese Through American Movies,A Place in the Sun,A Prairie Home Companion,A Prophet,A Pyromaniac's Love Story,A Raisin in the Sun,A River Runs Through It,A Room with a View,A Royal Affair,A Scanner Darkly,A Separation,A Serbian Film,A Serious Man,A Shock to the System,A Short Film About Killing,A Shot in the Dark,A Simple Plan,A Simple Twi

In [112]:
user_item.shape

(671, 8754)

In [163]:
pickle.dump(user_item, open('user_item.pkl', 'wb'))

In [6]:
user_item = pickle.load(open('user_item.pkl', 'rb')) 

In [17]:
# Calculate the user-user similarity matrix
user_similarity = cosine_similarity(user_item) 

In [18]:
user_similarity

array([[1.        , 0.        , 0.        , ..., 0.06291708, 0.        ,
        0.01746565],
       [0.        , 1.        , 0.12531695, ..., 0.02433832, 0.17199729,
        0.11410584],
       [0.        , 0.12531695, 1.        , ..., 0.08098382, 0.13660585,
        0.17019275],
       ...,
       [0.06291708, 0.02433832, 0.08098382, ..., 1.        , 0.04260878,
        0.08520194],
       [0.        , 0.17199729, 0.13660585, ..., 0.04260878, 1.        ,
        0.22867673],
       [0.01746565, 0.11410584, 0.17019275, ..., 0.08520194, 0.22867673,
        1.        ]])

In [28]:
user_similarity.shape

(671, 671)

In [19]:
pickle.dump(user_similarity, open('user_similarity.pkl', 'wb'))

In [7]:
user_similarity = pickle.load(open('user_similarity.pkl', 'rb'))

In [24]:
def user_based_recommendation():
    userId = int(input('Enter your User ID: '))
    similar_users = sorted(list(enumerate(user_similarity[userId - 1])), key=lambda x: x[1], reverse=True)
    recommendations = []
    item_index = user_item.loc[userId]
    unrated_movie = item_index[item_index == 0].index.to_list() #Movies which user has no interaction with
    # Recommend the top-rated movies from similar users
    for user, similarity_score in similar_users[1:4]:  #The first 3 similar users excluding the target user
        user = user + 1
        user_movies = user_item.loc[user]
        recommended_movies = user_movies[user_movies > 2].index.to_list() #Any movie rated above 2 by the similar users
        for movie in recommended_movies:
            recommendations.append(movie)
    recommendations = list(set(recommendations))
    rec_mov = [x for x in unrated_movie if x in recommendations]
    return rec_mov

In [32]:
user_based_output = user_based_recommendation()

Enter your User ID: 17


In [33]:
top_20_rec_user = pd.DataFrame({'Recommend Movies':user_based_output[:20]}, index=[i for i in range(1,21)])
top_20_rec_user

,Recommend Movies
1,10 Things I Hate About You
2,12 Angry Men
3,15 Minutes
4,21 Grams
5,25th Hour
6,300
7,8 Mile
8,8MM
9,A Bridge Too Far
10,A Bronx Tale


In [133]:
mov_poster(user_based_output)

**NB:** Recommendation based on similar users' interest to the target user's

# Ensemble Recommendation System

In [120]:
def recommend_movie():
    try:
        user_based = user_based_recommendation()
        movie_based = movie_based_recommendation()
        final_rec = [x for x in user_based if x in movie_based] #Prioritizing intersecting movie
        if final_rec == []:
            #If there's no intersection
            user_fin = user_based[:5]
            mov_fin = movie_based[:5]
            final_rec = user_fin + mov_fin
            shuffle(final_rec)
            return final_rec
        else:
            #If there is an intersection 
            final_rec = final_rec + [x for x in user_based if x not in movie_based]
            return final_rec[:10]
    except IndexError:
         pass

In [149]:
final_rec = recommend_movie()

Enter your User ID: 56
Enter the movie of your choice: batman


In [150]:
final_recommendation = pd.DataFrame({'Recommend Movies':final_rec}, index=[i for i in range(1, len(final_rec)+1)])
final_recommendation

,Recommend Movies
1,Batman Returns
2,Batman: Mask of the Phantasm
3,Batman: Under the Red Hood
4,Man of Steel
5,Superman
6,(500) Days of Summer
7,10 Cloverfield Lane
8,101 Dalmatians
9,12 Angry Men
10,13 Assassins


In [151]:
mov_poster(final_rec)